# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 33

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28,1990,26,26,2244,3,1,1,1,0,0,0.12,0.04,0.16,0.08,0.12,1.8,1.1,2.1,0.07,0.09,0.16,0.04,0.13,Matches
2,Max Aarons\Max-Aarons,eng ENG,DF,Norwich City,19,2000,31,31,2790,0,1,0,0,7,0,0.00,0.03,0.03,0.00,0.03,0.5,0.5,3.6,0.02,0.11,0.13,0.02,0.13,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21,1997,29,24,2049,13,3,0,0,2,0,0.57,0.13,0.70,0.57,0.70,13.5,13.5,2.5,0.59,0.11,0.70,0.59,0.70,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23,1996,25,9,825,1,2,0,0,0,0,0.11,0.22,0.33,0.11,0.33,2.4,2.4,1.0,0.26,0.11,0.37,0.26,0.37,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32,1987,11,9,875,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, engine='python')

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,33,29,2,2,72,25,47,89,62.0,34.5,27.5,0.83,W D W L W,53143,Mohamed Salah - 17,Alisson,NaN
2,Manchester City,33,21,3,9,81,34,47,66,78.1,31.7,46.4,1.41,W W L W L,54219,Sergio Agüero - 16,Ederson,NaN
3,Leicester City,33,17,7,9,63,31,32,58,52.8,39.2,13.7,0.41,W D D L W,32061,Jamie Vardy - 21,Kasper Schmeichel,NaN
4,Chelsea,33,17,6,10,60,44,16,57,59.0,33.0,26.0,0.79,W W W L W,40563,Tammy Abraham - 13,Kepa Arrizabalaga,NaN
5,Manchester Utd,33,15,10,8,56,33,23,55,51.4,31.6,19.8,0.60,W D W W W,72726,Marcus Rashford Anthony Martial - 15,David de Gea,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,220,100.0,100.0,4,69140,0,0,-4,4,44.2,335,114,0,2,3.5,3.0,7,Shkodran,3.0,14,0,40.1,320,112,False,231.2,264,106,1025,,2020-07-01T22:00:25.971699Z,51,0,0,0,69140.jpg,3.2,0,0,Mustafi,0.4,False,None,a,1,3,124.0,240,61,41,22593,1280,43598,355,0.6,8.0,Mustafi,1
2,0,4,187,100.0,100.0,4,98745,-1,1,-2,2,49.3,328,109,0,2,3.7,3.2,1,Héctor,3.2,15,1,29.2,347,123,False,170.8,311,121,983,,2019-12-09T20:00:21.228098Z,53,0,0,0,98745.jpg,3.2,0,0,Bellerín,1.1,False,None,a,1,3,70.0,306,99,41,195600,1301,141392,5726,0.6,7.7,Bellerín,2
3,2,1,292,100.0,100.0,4,111457,0,0,-3,3,164.8,206,52,0,2,3.3,2.8,6,Sead,2.8,21,0,46.8,294,100,False,239.0,258,104,1393,,2020-02-23T18:30:13.672943Z,52,0,0,0,111457.jpg,2.3,0,0,Kolasinac,0.5,False,None,a,1,3,67.0,307,100,49,62115,865,132303,285,0.5,9.4,Kolasinac,3
4,2,3,223,100.0,100.0,3,154043,0,0,-5,5,159.0,215,54,1,2,0.9,0.4,0,Ainsley,0.4,20,0,46.9,292,98,False,278.8,235,91,1277,,2019-09-22T18:00:10.824841Z,45,0,0,0,154043.jpg,2.2,1,0,Maitland-Niles,2.2,False,None,a,1,3,37.0,349,126,38,608028,613,648579,1484,0.1,8.4,Maitland-Niles,4
5,0,5,305,100.0,100.0,4,39476,0,0,-2,2,36.8,350,119,1,2,0.5,0.0,0,Sokratis,0.0,25,2,58.5,240,72,False,436.2,135,51,1696,,2020-06-18T18:00:15.974146Z,48,0,0,0,39476.jpg,3.0,0,0,Papastathopoulos,1.3,False,None,a,1,3,110.0,257,72,57,180776,237,226644,1654,0.0,11.9,Sokratis,6


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,33,29,2,2,72,25,47,89,62.0,34.5,27.5,0.83,W D W L W,53143,Mohamed Salah - 17,Alisson,NaN,1.045455,0.351532
2,Manchester City,33,21,3,9,81,34,47,66,78.1,31.7,46.4,1.41,W W L W L,54219,Sergio Agüero - 16,Ederson,NaN,0.960606,0.382661
3,Leicester City,33,17,7,9,63,31,32,58,52.8,39.2,13.7,0.41,W D D L W,32061,Jamie Vardy - 21,Kasper Schmeichel,NaN,1.187879,0.304867
4,Chelsea,33,17,6,10,60,44,16,57,59.0,33.0,26.0,0.79,W W W L W,40563,Tammy Abraham - 13,Kepa Arrizabalaga,NaN,1.000000,0.367879
5,Manchester Utd,33,15,10,8,56,33,23,55,51.4,31.6,19.8,0.60,W D W W W,72726,Marcus Rashford Anthony Martial - 15,David de Gea,NaN,0.957576,0.383822
6,Wolves,33,13,13,7,45,36,9,52,42.1,29.9,12.2,0.37,D W W W L,31360,Raúl Jiménez - 15,Rui Patrício,NaN,0.906061,0.404113
7,Arsenal,33,12,13,8,49,41,8,49,43.5,47.2,-3.8,-0.11,L L W W W,60279,Pierre-Emerick Aubameyang - 19,Bernd Leno,NaN,1.430303,0.239236
8,Tottenham,33,13,9,11,52,44,8,48,42.2,47.3,-5.1,-0.15,D D W L W,59384,Harry Kane - 13,Paulo Gazzaniga,NaN,1.433333,0.238513
9,Sheffield Utd,33,12,12,9,34,33,1,48,37.4,41.8,-4.4,-0.13,D L L W D,30869,Lys Mousset - 6,Dean Henderson,NaN,1.266667,0.281769


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    if df.loc[ix, 'element_type']<=2:
        # 0.24 is the probability to miss a penalty, which incurs -2 points
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] - 6 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

In [11]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.7622107969151671

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [12]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','garcia','chicharito','elliott']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'chicharito':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('javierhernandez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [13]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [14]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
191,Salah,28.873239,205,7.1,193.116808,6.688436
618,Fernandes,8.988764,80,8.9,58.156926,6.469958
281,McGovern,2.000000,11,5.5,12.669919,6.334959
215,De Bruyne,30.857143,216,7.0,194.715174,6.310214
192,Mané,29.850746,200,6.7,177.513801,5.946712
233,Rashford,25.932203,153,5.9,146.953573,5.666837
214,Sterling,28.269231,147,5.2,158.291000,5.599410
166,Vardy,30.000000,186,6.2,162.080206,5.402674
182,Alexander-Arnold,33.275862,193,5.8,176.377267,5.300457


In [15]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [16]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total xG: ' + str(total_xG))
print('Total goals: ' + str(total_goals))
print('Total xG / total goals: ' + str(total_xG/total_goals))

Total xG: 891.4000000000001
Total goals: 862
Total xG / total goals: 1.0341067285382832
